In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-06-15


Found rundate 2023-06-14 10:00:03 < 2023-06-15 at commit 84955ceb; returning commit 0e32d1d


'2023-06-15 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-06-15 10:00:02, NJSP was reporting 230 YTD deaths
Current YTD Deaths (2024-06-15 10:00:03-04:00): 294
Previous year YTD Deaths (adjusted; 2023-06-15 10:00:02): 230.75857915059075
Projected 2024 total: 719.3630093606299
27.4% change


FAUQStats cache miss: 2023, Thu Jun 15 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,Mulberry St,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,0.0,South Randolphville Rd,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,0.0,Outwater Ln,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,Washington St,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12500,03,Burlington,0333,Shamong Twsp,NaN,Atsion Rd,1.0,1.0,0.0,0.0,0.0,NaN,Atsion Rd,2023-06-11 12:29:00-04:00
12502,20,Union,2004,Elizabeth City,1,State Highway 1,1.0,0.0,0.0,0.0,1.0,NaN,NaN,2023-06-12 22:43:00-04:00
12504,01,Atlantic,0121,Somers Point City,NaN,Bethel Rd,1.0,1.0,0.0,0.0,0.0,NaN,Bethel Rd,2023-06-13 09:20:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,13,1,9,2,25
Bergen,9,3,9,0,20
Burlington,16,3,2,0,21
Camden,10,3,10,0,23
Cape May,1,0,0,0,1
Cumberland,6,2,2,0,9
Essex,8,3,10,0,21
Gloucester,7,2,0,0,9
Hudson,2,2,1,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

45.5% through the year, 54.5% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,7,18,13
Bergen,driver,9,20,9
Burlington,driver,5,26,16
Camden,driver,9,18,10
Cape May,driver,2,4,1
...,...,...,...,...
Salem,crashes,4,11,6
Somerset,crashes,3,21,5
Sussex,crashes,1,6,3


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,59,4,28,7,20
Bergen,41,1,20,9,15
Burlington,66,1,58,6,4
Camden,57,3,19,7,35
Cape May,4,0,3,1,1
Cumberland,22,0,17,5,3
Essex,60,1,20,7,33
Gloucester,24,1,18,4,2
Hudson,13,1,6,5,4


In [9]:
projected.to_csv(PROJECTED_CSV)